# User-based book Recommendation system
This notebook demonstrates a simple user-based recommendation system using Manhattan distance to find the nearest neighbor and recommend books. This system uses user-book ratings to make recommendations for other users.
Cell 2: Import Libraries

In [7]:
# import the necessary python libraries

import pandas as pd                    # for data manipulation and analysis
from tabulate import tabulate          # helps to present output in table format

In [8]:
# A dictionary containing user-book ratings
ReaderBookRatings = {
    'Abbey': {'The Exorcist':10, 'Dune':5, 'The Hobbit':9, 'Black Beauty':6}, 
    'Bob': {'Odyssey':8, 'Beloved':3, 'Dune':10, 'The Hobbit':5, 'Black Beauty':7}, 
    'Cody': {'Hamlet':7, 'Trust':4, 'The Exorcist':5, 'Dune':9, 'The Hobbit':1}, 
    'Dan': {'Beloved':5, 'Dune':7, 'The Hobbit':4}, 
    'Emily': {'Odyssey':3, 'Beloved':8, 'Dune':1, 'Black Beauty':7}, 
    'Fausa': {'Trust':3, 'The Exorcist':10, 'Dune':6, 'The Hobbit':10}
}

In [9]:
# manhattan distance funtion with two parameters (ratings1D, ratings2D)
def manhattanD(ratings1D, ratings2D):

    distance = 0
    # consider a book only if both users have rated it
    for book in ratings1D.keys():
        if book in ratings2D.keys():
            x = ratings1D[book]
            y = ratings2D[book]
            distance += pow(abs(x - y), 1)    # Add the absolute difference in a pair of ratings
    return pow(distance, 1/1)
         

In [10]:
# Create a dictionary to hold the Manhattan distances between each pair of readers

manhattan_distances = {}
for ReaderX in ReaderBookRatings:
    manhattan_distances[ReaderX] = {}
    for ReaderY in ReaderBookRatings:
        if ReaderX != ReaderY:
            # Calculate Manhattan distance between ReaderX and ReaderY
            manhattan_distances[ReaderX][ReaderY] = manhattanD(ReaderBookRatings[ReaderX], ReaderBookRatings[ReaderY])
        else:
            manhattan_distances[ReaderX][ReaderY] = '-'
        

In [11]:
# Convert the dictionary of distances to a pandas DataFrame for easier manipulation

similarity_matrix = pd.DataFrame(manhattan_distances)
print('\nSimilarity Matrix:')
print(tabulate(similarity_matrix, headers='keys', tablefmt='fancy_grid'))


Similarity Matrix:
╒═══════╤═════════╤═══════╤════════╤═══════╤═════════╤═════════╕
│       │ Abbey   │ Bob   │ Cody   │ Dan   │ Emily   │ Fausa   │
╞═══════╪═════════╪═══════╪════════╪═══════╪═════════╪═════════╡
│ Abbey │ -       │ 10.0  │ 17.0   │ 7.0   │ 5.0     │ 2.0     │
├───────┼─────────┼───────┼────────┼───────┼─────────┼─────────┤
│ Bob   │ 10.0    │ -     │ 5.0    │ 6.0   │ 19.0    │ 9.0     │
├───────┼─────────┼───────┼────────┼───────┼─────────┼─────────┤
│ Cody  │ 17.0    │ 5.0   │ -      │ 5.0   │ 8.0     │ 18.0    │
├───────┼─────────┼───────┼────────┼───────┼─────────┼─────────┤
│ Dan   │ 7.0     │ 6.0   │ 5.0    │ -     │ 9.0     │ 7.0     │
├───────┼─────────┼───────┼────────┼───────┼─────────┼─────────┤
│ Emily │ 5.0     │ 19.0  │ 8.0    │ 9.0   │ -       │ 5.0     │
├───────┼─────────┼───────┼────────┼───────┼─────────┼─────────┤
│ Fausa │ 2.0     │ 9.0   │ 18.0   │ 7.0   │ 5.0     │ -       │
╘═══════╧═════════╧═══════╧════════╧═══════╧═════════╧═════════╛


In [12]:
# # Find recommendations for each reader based on their nearest neighbour

recommendations = []
for ReaderX in ReaderBookRatings:
    min_distance = float('inf')
    nearest_neighbour = None

    # finding the nearest neighbour of each reader based on smallest value of manhattan distance
    for ReaderY in ReaderBookRatings:
        if ReaderX != ReaderY:
            distance = manhattan_distances[ReaderX][ReaderY]
            
            if distance != '-' and distance < min_distance:
                min_distance = distance
                nearest_neighbour = ReaderY

    # # Find books rated by the nearest neighbour of ReaderX but not by ReaderX
    recommended_books = []
    for book, rating in ReaderBookRatings[nearest_neighbour].items():
        
        if book not in ReaderBookRatings[ReaderX]:
            
            recommended_books.append((book, rating))

    recommendations.append({'Reader': ReaderX, 'Recommended Books': recommended_books})

# creating a recommendation table
recommendations_table = pd.DataFrame(recommendations)
print('\nRecommendations:')
print(tabulate(recommendations_table, headers='keys', showindex=False, tablefmt='fancy_grid'))



Recommendations:
╒══════════╤═══════════════════════════════════════════════════════╕
│ Reader   │ Recommended Books                                     │
╞══════════╪═══════════════════════════════════════════════════════╡
│ Abbey    │ [('Trust', 3)]                                        │
├──────────┼───────────────────────────────────────────────────────┤
│ Bob      │ [('Hamlet', 7), ('Trust', 4), ('The Exorcist', 5)]    │
├──────────┼───────────────────────────────────────────────────────┤
│ Cody     │ [('Odyssey', 8), ('Beloved', 3), ('Black Beauty', 7)] │
├──────────┼───────────────────────────────────────────────────────┤
│ Dan      │ [('Hamlet', 7), ('Trust', 4), ('The Exorcist', 5)]    │
├──────────┼───────────────────────────────────────────────────────┤
│ Emily    │ [('The Exorcist', 10), ('The Hobbit', 9)]             │
├──────────┼───────────────────────────────────────────────────────┤
│ Fausa    │ [('Black Beauty', 6)]                                 │
╘══════════╧════